In [1]:
import datetime
from datetime import datetime as dt
import requests
import xarray as xr 

In [2]:
def download_TAMSAT_day(query_date, save_path):

    query_month = query_date.strftime('%m')
    query_day = query_date.strftime('%d')

    URL_filename = str(query_date.year)+"/"+str(query_month)+"/rfe"+str(query_date.year)+"_"+str(query_month)+"_"+str(query_day)+".v3.1.nc"
    URL_full = "http://www.tamsat.org.uk/public_data/data/v3.1/daily/"+URL_filename

    response = requests.get(URL_full)
    save_filename = "TAMSAT_"+URL_filename.replace("/","_")
    open(os.path.join(save_path,save_filename), "wb").write(response.content)

In [3]:
def crop_and_save_TAMSAT_day(query_date, area, selected_area):

    query_month = query_date.strftime('%m')
    query_day = query_date.strftime('%d')
    URL_filename = str(query_date.year)+"/"+str(query_month)+"/rfe"+str(query_date.year)+"_"+str(query_month)+"_"+str(query_day)+".v3.1.nc"
    save_filename = "TAMSAT_"+URL_filename.replace("/","_")
    nc_file_content = xr.open_dataset(os.path.join(save_path,save_filename))

    xarray_variable_name = "rfe_filled"

    # cropping
    nc_file_content = nc_file_content.where((nc_file_content.lat < area[selected_area][0])
                            & (nc_file_content.lat > area[selected_area][2])
                            & (nc_file_content.lon > area[selected_area][1])
                            & (nc_file_content.lon < area[selected_area][3])
                        ).dropna(dim='lat', how='all').dropna(dim='lon', how='all')

    bT = nc_file_content[xarray_variable_name]
    bT = bT.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    bT.rio.crs
    bT.rio.write_crs("epsg:4326", inplace=True)
    output_filename = "TAMSAT_v3.1_"+selected_area+"_"+xarray_variable_name+"_"+str(query_date.year)+"_"+str(query_month)+"_"+str(query_day)+".tif"
    output_path = "../data/3_output/TAMSAT_v3.1_"+selected_area+"_"+xarray_variable_name+"/"

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    bT.rio.to_raster(os.path.join(output_path,output_filename))

## Download previous month

In [9]:
def download_TAMSAT_lastmonth(area=area, selected_area=selected_area, save_path=save_path):
    query_date = dt.today() - datetime.timedelta(days=8)

    for i in range(query_date.day):

        query_date_loop = query_date - datetime.timedelta(days=i)

        print("=== downloading TAMSAT for date",query_date_loop.date(),"===")
        download_TAMSAT_day(query_date_loop, save_path)
        crop_and_save_TAMSAT_day(query_date_loop, area, selected_area)

In [10]:
area = {
    'burkina': [16, -6, 9, 3],
    'west_africa':[29, -20, 3.5, 26]}
selected_area = "burkina"
save_path = "../data/0_downloads/"

download_TAMSAT_lastmonth()

=== downloading TAMSAT for date 2022-11-19 ===


ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

# Download whole year

In [7]:
area = {
    'burkina': [16, -6, 9, 3],
    'west_africa':[29, -20, 3.5, 26],
    'north_cameroon':[13.6,12,6.8,16],
    }
selected_area = "north_cameroon"
save_path = "../data/0_downloads/"

In [25]:
def download_TAMSAT_year(query_year, area=area, selected_area=selected_area, save_path=save_path):
    for i in range(365):

        query_date_loop = datetime.date(query_year,11,8) + datetime.timedelta(days=i)

        print("=== downloading TAMSAT for date",query_date_loop,"===")
        download_TAMSAT_day(query_date_loop, save_path)
        crop_and_save_TAMSAT_day(query_date_loop, area, selected_area)

In [26]:
download_TAMSAT_year(2020)

=== downloading TAMSAT for date 2020-11-08 ===
=== downloading TAMSAT for date 2020-11-09 ===
=== downloading TAMSAT for date 2020-11-10 ===
=== downloading TAMSAT for date 2020-11-11 ===
=== downloading TAMSAT for date 2020-11-12 ===
=== downloading TAMSAT for date 2020-11-13 ===
=== downloading TAMSAT for date 2020-11-14 ===
=== downloading TAMSAT for date 2020-11-15 ===
=== downloading TAMSAT for date 2020-11-16 ===
=== downloading TAMSAT for date 2020-11-17 ===
=== downloading TAMSAT for date 2020-11-18 ===
=== downloading TAMSAT for date 2020-11-19 ===
=== downloading TAMSAT for date 2020-11-20 ===
=== downloading TAMSAT for date 2020-11-21 ===
=== downloading TAMSAT for date 2020-11-22 ===
=== downloading TAMSAT for date 2020-11-23 ===
=== downloading TAMSAT for date 2020-11-24 ===
=== downloading TAMSAT for date 2020-11-25 ===
=== downloading TAMSAT for date 2020-11-26 ===
=== downloading TAMSAT for date 2020-11-27 ===
=== downloading TAMSAT for date 2020-11-28 ===
=== downloadi

KeyboardInterrupt: 